In [1]:
import pandas as pd
import mygene
import pathlib
import gseapy as gp
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import upsetplot as upset
from scipy import stats
from statsmodels.stats.multitest import multipletests

ConnectionError: HTTPConnectionPool(host='maayanlab.cloud', port=80): Max retries exceeded with url: /Enrichr/datasetStatistics (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000241A1EAFDF0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [44]:
manifest_path = "E:/YandexDisk/pydnameth/datasets/GPL13534/manifest"
manifest_pkl = f"{manifest_path}/manifest.pkl"
manifest = pd.read_pickle(manifest_pkl)
manifest['CHR'] = manifest['CHR'].str[3::]

In [8]:
path_cpg = "E:/YandexDisk/XAI/report2022/bio/"
cpg_1000_xlsx = f"{path_cpg}/1000.xlsx"
cpg_1000 = pd.read_excel(cpg_1000_xlsx)
cpg_1000_list = list(cpg_1000['features'])
genes_selected = set()
for cpg in cpg_1000_list:
    genes_raw = manifest.at[cpg, 'Gene']
    if isinstance(genes_raw, str):
        genes = genes_raw.split(';')
        genes_selected.update(set(genes))
if 'non-genic' in genes_selected:
    genes_selected.remove('non-genic')
if ' ' in genes_selected:
    genes_selected.remove(' ')
genes_selected = list(genes_selected)
genes_df = pd.DataFrame({'gene':genes_selected})
genes_df.to_excel(f"{path_cpg}/genes.xlsx", index=False)

In [14]:
mg = mygene.MyGeneInfo()
print(f"genes_selected: {len(genes_selected)}")
df_queries_all = []
genes_missed = []
number_of_synonyms = 0
for gene in genes_selected:
    df_query = mg.query(gene, scopes='entrezgene', species='human', as_dataframe=True)
    if df_query.empty:
        genes_missed.append(gene)
    else:
        df_queries_all.append(df_query)
        if gene not in set(df_query.loc[:, "symbol"].values):
            number_of_synonyms += 1
            print(f"{gene} not in {list(df_query.loc[:, 'symbol'].values)}")
print(f"Total number of synonyms: {number_of_synonyms}")

my_gene_all = pd.concat(df_queries_all)
my_gene_all.to_excel(f"{path_cpg}/my_gene_all.xlsx", index=True)

genes_missed_df = pd.DataFrame({'gene': genes_missed})
genes_missed_df.to_excel(f"{path_cpg}/genes_mygene_missed.xlsx", index=False)

genes_selected_all = list(set(my_gene_all.loc[:, "symbol"].values))
genes_selected_all_df = pd.DataFrame({'gene': genes_selected_all})
genes_selected_all_df.to_excel(f"{path_cpg}/genes_mygene_all.xlsx", index=False)

genes_selected: 800
TMEM206 not in ['PACC1']
KILLIN not in ['KLLN']
CD3EAP not in ['POLR1G']
C18orf1 not in ['LDLRAD4']
ICK not in ['CILK1']
WDR51B not in ['POC1B']
PRKCDBP not in ['CAVIN3']
C17orf90 not in ['OXLD1']
BAT4 not in ['GPANK1']
C12orf34 not in ['FAM222A']
TMEM84 not in ['EWSAT1']
DPCR1 not in ['MUCL3']
BAI1 not in ['BAIAP3', 'ADGRB1', 'LOC100420499', 'BAIAP2L2', 'BAIAP2L1', 'ADGRB3', 'MAGI1', 'BAIAP2']
TSSC1 not in ['EIPR1-IT1', 'EIPR1']
C5orf33 not in ['NADK2']
C1orf107 not in ['UTP25']
FAM46C not in ['TENT5C']
CCDC75 not in ['GPATCH11']
SEC1 not in ['SCFD2', 'SCFD1', 'STXBP1', 'STXBP3', 'SEC1P', 'STXBP2', 'STXBP5', 'VPS33A', 'VPS45']
TMEM90A not in ['SYNDIG1L']
PCNXL2 not in ['PCNX2']
C13orf23 not in ['PROSER1']
SEPT9 not in ['SEPTIN9']
C9orf3 not in ['MSANTD3-TMEFF1', 'AOPEP']
KDELC1 not in ['POGLUT2P1', 'POGLUT2']
ACPT not in ['ACP4']
RP11-529I10.4 not in ['DPCD']
FAM102A not in ['EEIG1', 'LOC106783495']
CUGBP2 not in ['CELF2']
BAI2 not in ['ADGRB3', 'ADGRB2', 'ADGRB1']

In [20]:
pathlib.Path(f"{path_cpg}/GSEA").mkdir(parents=True, exist_ok=True)
libraries = gp.get_library_name("Human")
df_libraries = pd.DataFrame(index=libraries)
df_libraries.to_excel(f"{path_cpg}/GSEA/libraries.xlsx", index=True)

genes_dict_of_lists = {
    "origin": genes_selected,
    "mygene_all": genes_selected_all
}

for genes in genes_dict_of_lists:
    dfs_enrichr = []
    for genes_list in libraries:
        pathlib.Path(f"{path_cpg}/GSEA/{genes}/{genes_list}").mkdir(parents=True, exist_ok=True)
        df_enrichr = gp.enrichr(
            gene_list=genes_dict_of_lists[genes],
            gene_sets=genes_list,
            organism='Human',
            outdir=f"{path_cpg}/GSEA/{genes}/{genes_list}",
            cutoff=1.00,
            verbose=True,
            no_plot=True
        )
        dfs_enrichr.append(df_enrichr.results)
    dfs_enrichr = pd.concat(dfs_enrichr)
    dfs_enrichr.to_excel(f"{path_cpg}/GSEA/{genes}/results.xlsx", index=True)
    dfs_enrichr.to_pickle(f"{path_cpg}/GSEA/{genes}/results.pkl")

2022-12-05 21:01:57,568 Connecting to Enrichr Server to get latest library names
2022-12-05 21:01:58,698 Analysis name: , Enrichr Library: ARCHS4_Cell-lines
2022-12-05 21:02:03,427 Save file of enrichment results: Job Id:8d0b24139860a0abde38002435f6ccd7
2022-12-05 21:02:03,432 Done.

2022-12-05 21:02:03,434 Connecting to Enrichr Server to get latest library names
2022-12-05 21:02:04,535 Analysis name: , Enrichr Library: ARCHS4_IDG_Coexp
2022-12-05 21:02:08,583 Save file of enrichment results: Job Id:e3ee5b65a14f9a62d46dfcf9ccf12d2d
2022-12-05 21:02:08,588 Done.

2022-12-05 21:02:08,590 Connecting to Enrichr Server to get latest library names
2022-12-05 21:02:09,684 Analysis name: , Enrichr Library: ARCHS4_Kinases_Coexp
2022-12-05 21:02:13,809 Save file of enrichment results: Job Id:70164a1dfb0d7dfaab0aa30403e946ae
2022-12-05 21:02:13,815 Done.

2022-12-05 21:02:13,817 Connecting to Enrichr Server to get latest library names
2022-12-05 21:02:14,916 Analysis name: , Enrichr Library: ARCH

In [27]:
libraries_file = [
    "libraries_target_GO_Biological_Process",
    "libraries_target_GO_Cellular_Component",
    "libraries_target_GO_Molecular_Function",
    "libraries_target_nonGO",
]

for library_file in libraries_file:
    libraries_target = pd.read_excel(f"{path_cpg}/GSEA/{library_file}.xlsx")["library"].values

    gsea_cols = ["Gene_set", "Term", "Overlap", "P-value", "Adjusted P-value", "Odds Ratio", "Combined Score"]
    for genes in ["origin"]:

        dfs_enrichr = pd.read_pickle(f"{path_cpg}/GSEA/{genes}/results.pkl")
        dfs_enrichr = dfs_enrichr.loc[(dfs_enrichr["Adjusted P-value"] < 0.05) & (dfs_enrichr["Gene_set"].isin(libraries_target)), gsea_cols]
        dfs_enrichr.index = range(len(dfs_enrichr))

        if dfs_enrichr.empty == False:
            dfs_enrichr[r'$ -\log_{10}(\mathrm{p-value})$'] = -np.log10(dfs_enrichr.loc[:, 'Adjusted P-value'].values)
            dfs_enrichr.rename(columns={'Gene_set': 'Gene Library'}, inplace=True)
            dfs_enrichr.to_excel(f"{path_cpg}/GSEA/{genes}/terms_{library_file}.xlsx")
            plt.figure(figsize=(10, 0.5 * dfs_enrichr.shape[0]))
            sns.set_theme(style='whitegrid', font_scale=2)
            bar = sns.barplot(
                data=dfs_enrichr,
                hue="Gene Library",
                y=dfs_enrichr.index,
                x=r'$ -\log_{10}(\mathrm{p-value})$',
                palette=list(px.colors.qualitative.Alphabet) + list(px.colors.qualitative.Dark24) + list(px.colors.qualitative.Light24),
                edgecolor='black',
                orient="h",
                dodge=False
            )
            bar.set_yticklabels(dfs_enrichr["Term"])
            sns.move_legend(bar, "upper left", bbox_to_anchor=(1, 1))
            plt.savefig(f"{path_cpg}/GSEA/{genes}/terms_{library_file}.png", bbox_inches='tight')
            plt.savefig(f"{path_cpg}/GSEA/{genes}/terms_{library_file}.pdf", bbox_inches='tight')
            plt.close()

In [ ]:
genes = "origin"

df_upset_terms = pd.read_excel(f"{path_cpg}GSEA/enrichr.xlsx")
dict_upset_gene_lists = {"Parkinson": genes_selected}
for ind, row in df_upset_terms.iterrows():
    print(f"{row['library']} {row['term']}")
    library_dict = gp.parser.get_library(row['library'], organism='Human')
    if f"{row['code']}" not in dict_upset_gene_lists:
        dict_upset_gene_lists[f"{row['code']}"] = library_dict[row['term']]
    else:
        dict_upset_gene_lists[f"{row['code']}"] = list(set(dict_upset_gene_lists[f"{row['code']}"]).union(set(library_dict[row['term']])))

In [35]:
upset_genes_all = list(set().union(*list(dict_upset_gene_lists.values())))
df_upset = pd.DataFrame(index=upset_genes_all)
for k, v in dict_upset_gene_lists.items():
    df_upset[k] = df_upset.index.isin(v)
df_upset = df_upset.set_index(list(dict_upset_gene_lists.keys()))
tmp = plt.figure(figsize=(85, 15))
upset_fig = upset.UpSet(df_upset, subset_size='count', show_counts=True, min_degree=1, element_size=None, totals_plot_elements=5).plot(tmp)
plt.savefig(f"{path_cpg}/GSEA/{genes}/upset.png", bbox_inches='tight')
plt.savefig(f"{path_cpg}/GSEA/{genes}/upset.pdf", bbox_inches='tight')
plt.close()

In [46]:
cpg_411761_list = list(pd.read_excel(f"{path_cpg}/411761.xlsx")['features'])

In [47]:
pathlib.Path(f"{path_cpg}/region_enrichment").mkdir(parents=True, exist_ok=True)
pval_show_type = "cross" # "color"
orders = {
    'CHR': [str(x) for x in range(1, 24)],
    'RELATION_TO_UCSC_CPG_ISLAND': ['S_Shelf', 'S_Shore', 'Island', 'N_Shore', 'N_Shelf', 'OpenSea'],
    'UCSC_REFGENE_GROUP': ['TSS1500', 'TSS200', '5\'UTR', '1stExon', 'Body', '3\'UTR']
}
col_names = {
    'CHR': "CHR",
    'RELATION_TO_UCSC_CPG_ISLAND': "Relation_to_Island",
    'UCSC_REFGENE_GROUP': "UCSC_RefGene_Group"
}
fig_sizes = {
    'CHR': (17, 10),
    'RELATION_TO_UCSC_CPG_ISLAND': (5, 10),
    'UCSC_REFGENE_GROUP': (5, 10)
}
colors = {
    'CHR': px.colors.qualitative.Dark24,
    'RELATION_TO_UCSC_CPG_ISLAND': px.colors.qualitative.Light24[17:23],
    'UCSC_REFGENE_GROUP': px.colors.qualitative.Light24[11:17]
}
df_fisher_target = manifest.loc[cpg_1000_list, :]
df_fisher_global = manifest.loc[cpg_411761_list, :]
df_fisher_padding = df_fisher_global.loc[~df_fisher_global.index.isin(cpg_1000_list), :]
for var in orders:
    columns=["11", "12", "21", "22", "sum", "pval", "odds_ratio"]
    df_var = pd.DataFrame(index=orders[var], columns=columns, data=np.zeros((len(orders[var]), len(columns))))
    df_var.index.name = col_names[var].replace("_", " ")
    for var_val in orders[var]:
        contingency_table = pd.DataFrame(index=["specific", "non-specific"], columns=["in_val", "not_in_val"])
        contingency_table.at["specific", "in_val"] = df_fisher_target.loc[df_fisher_target[col_names[var]] == var_val, :].shape[0]
        contingency_table.at["specific", "not_in_val"] = df_fisher_target.loc[df_fisher_target[col_names[var]] != var_val, :].shape[0]
        contingency_table.at["non-specific", "in_val"] = df_fisher_padding.loc[df_fisher_padding[col_names[var]] == var_val, :].shape[0]
        contingency_table.at["non-specific", "not_in_val"] = df_fisher_padding.loc[df_fisher_padding[col_names[var]] != var_val, :].shape[0]
        df_var.at[var_val, "11"] = contingency_table.at["specific", "in_val"]
        df_var.at[var_val, "12"] = contingency_table.at["specific", "not_in_val"]
        df_var.at[var_val, "21"] = contingency_table.at["non-specific", "in_val"]
        df_var.at[var_val, "22"] = contingency_table.at["non-specific", "not_in_val"]
        df_var.at[var_val, "sum"] = contingency_table.values.sum()
        odds_ratio, pval = stats.fisher_exact(contingency_table.to_numpy(), alternative='two-sided')
        if np.isnan(odds_ratio):
            odds_ratio = 1.0
        df_var.at[var_val, "odds_ratio"], df_var.at[var_val, "pval"] = odds_ratio, pval
    _, df_var['pval_fdr_bh'], _, _ = multipletests(df_var['pval'].values, 0.05, method='fdr_bh')
    df_var[r'$ \log_{10}(\mathrm{Odds\ ratio})$'] = np.log10(df_var.loc[:, 'odds_ratio'].values)
    df_var[r'$ -\log_{10}(\mathrm{p-value})$'] = -np.log10(df_var.loc[:, 'pval_fdr_bh'].values)
    df_var.to_excel(f"{path_cpg}/region_enrichment/fisher_{var}.xlsx")

    plt.figure(figsize=fig_sizes[var])
    plt.xticks(rotation=90)
    sns.set_theme(style='whitegrid', font_scale=2)
    if pval_show_type == "color":
        plot = plt.scatter(df_var.index, df_var.loc[:, r'$ \log_{10}(\mathrm{Odds\ ratio})$'].values, c=df_var.loc[:, r'$ -\log_{10}(\mathrm{p-value})$'].values, cmap='Reds')
        plt.clf()
        cbar = plt.colorbar(plot)
        plt.xticks(rotation=90)
        cbar.set_label(r"$-\log_{10}(\mathrm{p-value})$", horizontalalignment='center')
        ax = sns.barplot(data=df_var, x=df_var.index, y=r'$ \log_{10}(\mathrm{Odds\ ratio})$', hue=r'$ -\log_{10}(\mathrm{p-value})$', palette='Reds', dodge=False, edgecolor='black')
        ax.legend_.remove()
    else:
        bar = sns.barplot(data=df_var, x=df_var.index, y=r'$ \log_{10}(\mathrm{Odds\ ratio})$', palette=colors[var], edgecolor='black')
        for bar_index, this_bar in enumerate(bar.patches):
            if df_var.at[df_var.index[bar_index], "pval_fdr_bh"] < 0.05:
                this_bar.set_hatch('x')
            this_bar.set_edgecolor('skyblue')
    plt.savefig(f"{path_cpg}/region_enrichment/fisher_{var}.png", bbox_inches='tight')
    plt.savefig(f"{path_cpg}/region_enrichment/fisher_{var}.pdf", bbox_inches='tight')
    plt.close()